# Tarea 05 - Imágenes Generativas
## Detección de Anomalías en Imágenes Industriales

Este notebook implementa un sistema de detección de anomalías usando el dataset MVTec AD.

**Objetivo**: Implementar modelos de autoencoder (clásico y U-net) para la reconstrucción de imágenes y detección de anomalías, utilizando:
- **Pytorch Lightning** para la estructuración del entrenamiento
- **Hydra** para la gestión modular de configuraciones
- **WandB** para el monitoreo y comparación de experimentos

**Estructura del Notebook**:
1. **Configuración inicial**: Instalación de dependencias y montaje de Google Drive
2. **Definición de modelos**: Arquitecturas de autoencoders (clásico y U-net)
3. **Módulo Lightning**: Funciones de pérdida y callbacks de visualización
4. **Configuración Hydra**: Gestión modular de hiperparámetros
5. **Carga de datos**: Dataset MVTec AD con transformaciones
6. **Entrenamiento**: Ejecución de experimentos (automatizada o individual)
7. **Evaluación**: Métricas y visualizaciones de resultados


En la siguiente celda se instalan todas las librerías necesarias para el proyecto:
- PyTorch y TorchVision para el manejo de redes neuronales
- Pytorch Lightning para la estructuración del entrenamiento
- Hydra para la gestión de configuraciones
- WandB para el monitoreo de experimentos
- Otras librerías auxiliares (matplotlib, scikit-learn, etc.)

In [5]:
## 0. Configuración Inicial

### 0.1. Instalación de Dependencias
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install matplotlib numpy scikit-learn opencv-python pillow tqdm wandb pytorch-lightning hydra-core omegaconf


Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 31.7 MB/s eta 0:00:00


Esta celda monta Google Drive para acceder a los archivos del proyecto (dataset, configuraciones, checkpoints, etc.).

In [6]:
### 0.2. Montar Google Drive

from google.colab import drive
drive.mount('/content/drive')


SyntaxError: invalid syntax (ipython-input-2540786305.py, line 3)

## 0.4. Definición de Arquitecturas de Modelos

En esta sección se definen las dos arquitecturas de autoencoders que se utilizarán en el proyecto:

- **Autoencoder Clásico**: Arquitectura tradicional sin skip connections
  - Encoder: Convoluciones con BatchNorm y ReLU
  - Decoder: Transconvoluciones con BatchNorm, ReLU y Tanh en la salida
  - Función `encode()`: Extrae el vector latente
  
- **U-Net Autoencoder**: Arquitectura con skip connections para mejorar la reconstrucción
  - Encoder: Convoluciones con skip connections guardadas
  - Decoder: Transconvoluciones que concatenan con skip connections del encoder
  - Función `encode()`: Extrae el vector latente y las skip connections

**Características comunes**:
- Imágenes de entrada: 128x128x3
- Espacio latente configurable (por defecto: 128 canales)
- Salida: 128x128x3 con valores en [-1, 1] (gracias a Tanh)


In [ ]:
"""
Módulo con las arquitecturas de autoencoders para detección de anomalías
"""
import torch
import torch.nn as nn


class AutoencoderClassic(nn.Module):
    """Autoencoder clásico sin skip connections"""

    def __init__(self, input_channels=3, latent_dim=128, encoder_channels=None, decoder_channels=None, architecture=None):
        super(AutoencoderClassic, self).__init__()
        self.architecture = architecture  # Store for logging purposes

        if encoder_channels is None:
            encoder_channels = [64, 128, 256, 512]
        if decoder_channels is None:
            decoder_channels = [512, 256, 128, 64]

        # Encoder
        encoder_layers = []
        in_channels = input_channels

        for out_channels in encoder_channels:
            encoder_layers.extend([
                nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(out_channels)
            ])
            in_channels = out_channels

        # Capa final del encoder
        encoder_layers.extend([
            nn.Conv2d(in_channels, latent_dim, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        ])

        self.encoder = nn.Sequential(*encoder_layers)

        # Decoder
        decoder_layers = []
        in_channels = latent_dim

        for out_channels in decoder_channels:
            decoder_layers.extend([
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(out_channels)
            ])
            in_channels = out_channels

        # Capa final del decoder
        decoder_layers.extend([
            nn.ConvTranspose2d(in_channels, input_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        ])

        self.decoder = nn.Sequential(*decoder_layers)

    def encode(self, x):
        """Extrae el vector latente de la entrada"""
        return self.encoder(x)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


class UNetAutoencoder(nn.Module):
    """Autoencoder U-net con skip connections"""

    def __init__(self, input_channels=3, latent_dim=128, encoder_channels=None, decoder_channels=None, architecture=None):
        super(UNetAutoencoder, self).__init__()
        self.architecture = architecture  # Store for logging purposes

        if encoder_channels is None:
            encoder_channels = [64, 128, 256, 512]
        if decoder_channels is None:
            decoder_channels = [512, 256, 128, 64]

        # Encoder con skip connections
        self.encoder_blocks = nn.ModuleList()
        in_channels = input_channels

        for out_channels in encoder_channels:
            self.encoder_blocks.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels)
                )
            )
            in_channels = out_channels

        # Capa bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv2d(in_channels, latent_dim, kernel_size=4, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder con skip connections
        self.decoder_blocks = nn.ModuleList()
        in_channels = latent_dim

        # Primera capa del decoder (sin skip connection)
        self.decoder_blocks.append(
            nn.Sequential(
                nn.ConvTranspose2d(in_channels, decoder_channels[0], kernel_size=4, stride=2, padding=1),
                nn.ReLU(),
                nn.BatchNorm2d(decoder_channels[0])
            )
        )
        in_channels = decoder_channels[0]

        # Resto de capas del decoder con skip connections
        for i, out_channels in enumerate(decoder_channels[1:], 1):
            # Duplicar canales de entrada para concatenar con skip connection
            self.decoder_blocks.append(
                nn.Sequential(
                    nn.ConvTranspose2d(in_channels * 2, out_channels, kernel_size=4, stride=2, padding=1),
                    nn.ReLU(),
                    nn.BatchNorm2d(out_channels)
                )
            )
            in_channels = out_channels

        # Capa final
        self.final_layer = nn.Sequential(
            nn.ConvTranspose2d(in_channels * 2, input_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh()
        )

    def encode(self, x):
        """Extrae el vector latente de la entrada"""
        # Encoder - guardar skip connections
        skip_connections = []
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
            skip_connections.append(x)

        # Bottleneck
        x = self.bottleneck(x)
        return x, skip_connections

    def forward(self, x):
        # Encoder - guardar skip connections
        skip_connections = []
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
            skip_connections.append(x)

        # Bottleneck
        x = self.bottleneck(x)

        # Decoder - usar skip connections
        # Primera capa del decoder (sin skip connection)
        x = self.decoder_blocks[0](x)

        # Resto de capas del decoder con skip connections
        # Las skip connections están en orden: [64, 128, 256, 512] (índices 0, 1, 2, 3)
        # Necesitamos usarlas en orden inverso: [512, 256, 128, 64]
        # decoder_blocks[1:] son los bloques que necesitan skip connections
        # Para decoder_blocks[1] (segundo bloque): necesita skip_connections[-1] = 512
        # Para decoder_blocks[2] (tercer bloque): necesita skip_connections[-2] = 256
        # Para decoder_blocks[3] (cuarto bloque): necesita skip_connections[-3] = 128
        for i, decoder_block in enumerate(self.decoder_blocks[1:], start=1):
            # Obtener skip connection correspondiente (en orden inverso)
            # i va de 1 a 3, necesitamos índices -1, -2, -3
            skip_idx = -i  # Para i=1: -1 (512), para i=2: -2 (256), para i=3: -3 (128)
            skip = skip_connections[skip_idx]

            # Asegurar que las dimensiones espaciales coincidan
            if x.shape[2:] != skip.shape[2:]:
                x = nn.functional.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)

            # Concatenar con skip connection
            x = torch.cat([x, skip], dim=1)
            x = decoder_block(x)

        # Capa final con último skip connection (skip_connections[0] = 64)
        skip = skip_connections[0]
        if x.shape[2:] != skip.shape[2:]:
            x = nn.functional.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip], dim=1)
        x = self.final_layer(x)

        return x

print("✓ Arquitecturas de modelos definidas correctamente")


### Módulo de Pytorch Lightning

En esta sección se define el módulo de Lightning que encapsula:
- **Funciones de pérdida**: L1, L2, SSIM, SSIM+L1
- **AutoencoderLightning**: Clase que gestiona el entrenamiento, validación y logging automático
- **Callback de Visualización**: Genera visualizaciones periódicas durante el entrenamiento (reconstrucciones y t-SNE)


In [ ]:
"""
Módulo de Pytorch Lightning para el entrenamiento de autoencoders
"""
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torchmetrics import StructuralSimilarityIndexMeasure
import torch.nn.functional as F


class LossFunctions:
    """Funciones de pérdida para el entrenamiento"""

    @staticmethod
    def l1_loss(pred, target):
        return F.l1_loss(pred, target)

    @staticmethod
    def l2_loss(pred, target):
        return F.mse_loss(pred, target)

    @staticmethod
    def ssim_loss(pred, target):
        # data_range=2.0 porque las imágenes están normalizadas a [-1, 1]
        ssim = StructuralSimilarityIndexMeasure(data_range=2.0)
        ssim_val = ssim(pred, target)
        return 1 - ssim_val  # SSIM es una métrica de similitud, convertimos a pérdida

    @staticmethod
    def ssim_l1_loss(pred, target, alpha=0.5):
        ssim = LossFunctions.ssim_loss(pred, target)
        l1 = LossFunctions.l1_loss(pred, target)
        return alpha * ssim + (1 - alpha) * l1


class AutoencoderLightning(pl.LightningModule):
    """Módulo de Lightning para entrenar autoencoders"""

    def __init__(self, model, learning_rate=0.001, loss_function="L2", scheduler_config=None):
        super().__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.loss_function = loss_function
        self.scheduler_config = scheduler_config or {"step_size": 10, "gamma": 0.5}

        # Inicializar función de pérdida
        if loss_function == "L1":
            self.criterion = LossFunctions.l1_loss
        elif loss_function == "L2":
            self.criterion = LossFunctions.l2_loss
        elif loss_function == "SSIM":
            self.criterion = LossFunctions.ssim_loss
        elif loss_function == "SSIM_L1":
            self.criterion = LossFunctions.ssim_l1_loss
        else:
            raise ValueError(f"Función de pérdida no reconocida: {loss_function}")

        # Métricas
        # data_range=2.0 porque las imágenes están normalizadas a [-1, 1]
        self.ssim_metric = StructuralSimilarityIndexMeasure(data_range=2.0)

        # Guardar pérdidas de entrenamiento
        self.train_losses = []

        # Guardar hiperparámetros
        self.save_hyperparameters(ignore=['model'])

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x = batch
        x_recon = self(x)
        loss = self.criterion(x_recon, x)

        # Logging
        self.log('train/loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train/learning_rate', self.optimizers().param_groups[0]['lr'], on_step=True)

        return loss

    def on_train_epoch_end(self):
        # Guardar pérdida promedio de la época
        epoch_loss = self.trainer.callback_metrics.get('train/loss_epoch', None)
        if epoch_loss is not None:
            self.train_losses.append(epoch_loss.item())

    def validation_step(self, batch, batch_idx):
        x = batch
        x_recon = self(x)
        loss = self.criterion(x_recon, x)

        # Calcular SSIM
        ssim_val = self.ssim_metric(x_recon, x)

        # Logging
        self.log('val/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log('val/ssim', ssim_val, on_step=False, on_epoch=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.StepLR(
            optimizer,
            step_size=self.scheduler_config.get("step_size", 10),
            gamma=self.scheduler_config.get("gamma", 0.5)
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch"
            }
        }

print("✓ Módulo de Lightning definido correctamente")

# Callback para visualizaciones periódicas durante el entrenamiento
from pytorch_lightning.callbacks import Callback
from sklearn.manifold import TSNE

class VisualizationCallback(Callback):
    """Callback para generar visualizaciones periódicas durante el entrenamiento"""

    def __init__(self, val_loader, device, log_every_n_epochs=5, num_samples=16):
        """
        Args:
            val_loader: DataLoader del set de validación
            device: Dispositivo (cuda/cpu)
            log_every_n_epochs: Cada cuántas épocas generar visualizaciones
            num_samples: Número de imágenes a visualizar
        """
        super().__init__()
        self.val_loader = val_loader
        self.device = device
        self.log_every_n_epochs = log_every_n_epochs
        self.num_samples = num_samples

    def on_train_epoch_end(self, trainer, pl_module):
        """Genera visualizaciones al final de cada N épocas"""
        current_epoch = trainer.current_epoch

        # Solo generar visualizaciones cada N épocas
        if current_epoch % self.log_every_n_epochs == 0 or current_epoch == trainer.max_epochs - 1:
            pl_module.eval()

            # Función auxiliar para denormalizar
            def denormalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
                for t, m, s in zip(tensor, mean, std):
                    t.mul_(s).add_(m)
                return tensor.clamp_(0, 1)

            val_images = []
            val_reconstructions = []
            val_latent_vectors = []

            with torch.no_grad():
                for batch_idx, images in enumerate(self.val_loader):
                    images = images.to(self.device)

                    # Obtener reconstrucciones
                    reconstructed = pl_module.model(images)

                    # Extraer vectores latentes
                    if hasattr(pl_module.model, 'encode'):
                        latent = pl_module.model.encode(images)
                        # Para UNet, encode devuelve (latent, skip_connections), solo necesitamos latent
                        if isinstance(latent, tuple):
                            latent = latent[0]
                        # Aplanar el vector latente para t-SNE
                        latent_flat = latent.view(latent.size(0), -1)
                        val_latent_vectors.append(latent_flat.cpu().numpy())

                    # Guardar imágenes para visualización (hasta num_samples)
                    if len(val_images) < self.num_samples:
                        remaining = self.num_samples - len(val_images)
                        val_images.append(images[:remaining].cpu())
                        val_reconstructions.append(reconstructed[:remaining].cpu())

                    if len(val_images) >= self.num_samples:
                        break

            # Concatenar todas las imágenes
            if val_images:
                val_images = torch.cat(val_images[:self.num_samples])[:self.num_samples]
                val_reconstructions = torch.cat(val_reconstructions[:self.num_samples])[:self.num_samples]

                # Visualizar reconstrucciones del set de validación
                import matplotlib.pyplot as plt
                import numpy as np
                import wandb

                fig, axes = plt.subplots(4, 8, figsize=(20, 10))
                axes = axes.flatten()
                val_reconstruction_images = []

                for i in range(min(self.num_samples, len(val_images))):
                    # Imagen original
                    img = denormalize(val_images[i].clone())
                    axes[i*2].imshow(img.permute(1, 2, 0))
                    axes[i*2].set_title(f'Original {i+1}')
                    axes[i*2].axis('off')

                    # Reconstrucción
                    recon = denormalize(val_reconstructions[i].clone())
                    axes[i*2+1].imshow(recon.permute(1, 2, 0))
                    axes[i*2+1].set_title(f'Reconstrucción {i+1}')
                    axes[i*2+1].axis('off')

                    # Preparar para wandb
                    img_np = (img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
                    recon_np = (recon.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
                    val_reconstruction_images.append(wandb.Image(img_np, caption=f"Original Val {i+1}"))
                    val_reconstruction_images.append(wandb.Image(recon_np, caption=f"Reconstrucción Val {i+1}"))

                plt.tight_layout()

                # Loggear reconstrucciones en wandb
                if trainer.logger:
                    trainer.logger.experiment.log({
                        f"val/reconstructions_epoch_{current_epoch}": val_reconstruction_images,
                    })

                plt.close(fig)

                # Visualización t-SNE del espacio latente
                if val_latent_vectors:
                    all_latent_vectors = np.concatenate(val_latent_vectors, axis=0)

                    # Aplicar t-SNE (reducir a 2D para visualización)
                    perplexity_val = min(30, len(all_latent_vectors) - 1)
                    if perplexity_val > 0:
                        tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_val)
                        latent_2d = tsne.fit_transform(all_latent_vectors)

                        # Visualizar t-SNE
                        fig, ax = plt.subplots(figsize=(10, 8))
                        ax.scatter(latent_2d[:, 0], latent_2d[:, 1], alpha=0.6, s=50)
                        ax.set_title(f't-SNE del Espacio Latente (Época {current_epoch})')
                        ax.set_xlabel('Componente t-SNE 1')
                        ax.set_ylabel('Componente t-SNE 2')
                        ax.grid(True, alpha=0.3)
                        plt.tight_layout()

                        # Loggear t-SNE en wandb
                        if trainer.logger:
                            trainer.logger.experiment.log({
                                f"val/tsne_latent_epoch_{current_epoch}": wandb.Image(fig),
                            })

                        plt.close(fig)

            pl_module.train()

print("✓ Callback de visualización definido correctamente")


## 0.3. Configuración con Hydra

En esta sección se configura el proyecto usando **Hydra** para la gestión modular de configuraciones, cumpliendo con los requisitos del enunciado.

**Estructura de configuración (según enunciado)**:
```
conf/
├── config.yaml          # Configuración principal
├── model/               # Configuraciones de modelos
│   ├── autoencoder_classic.yaml
│   └── unet.yaml
├── trainer/             # Configuración del entrenamiento
│   └── default.yaml
└── logger/              # Configuración de WandB
    └── wandb.yaml
```

**Características**:
- ✅ Configuración centralizada en archivos YAML
- ✅ Separación modular de hiperparámetros (modelo, entrenamiento, logger)
- ✅ Permite cambiar hiperparámetros fácilmente usando overrides
- ✅ Integración con WandB para tracking de experimentos
- ✅ Permite ejecutar experimentos con distintos parámetros (dimensión latente, épocas, batch size, etc.)

**Ubicación de archivos**:
- Los archivos de configuración pueden estar en el directorio actual (`/content/conf`) o en Google Drive
- El código busca automáticamente en ambas ubicaciones


### 0.3.1. Inicialización de Hydra y Configuración
La siguiente celda:
- Inicializa Hydra con la configuración del proyecto
- Crea los archivos de configuración si no existen
- Registra las clases del notebook para que Hydra pueda instanciarlas
- Carga la configuración base del proyecto
- Configura la autenticación de WandB

**Importante**: La siguiente celda debe ejecutarse antes de cualquier otra que use la configuración de Hydra.

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
import cv2
import hydra
from omegaconf import DictConfig, OmegaConf
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

%pip show hydra-core
%pip install --upgrade hydra-core

# Los módulos ya están definidos en las celdas anteriores
# No es necesario importarlos desde archivos externos

# Autenticación de Weights & Biases
import wandb
wandb.login()

# Inicializar Hydra para ejecutar múltiples configuraciones
# En Colab, usamos hydra.initialize() en lugar del decorador @hydra.main
# IMPORTANTE: Hydra requiere que config_path sea una ruta RELATIVA, no absoluta
# Los archivos de configuración deben estar en el directorio de trabajo o en Google Drive

# Configuración: Usar solo Google Drive
# Definir ruta base de Google Drive
DRIVE_BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/Tarea5-IA'
drive_conf_dir = os.path.join(DRIVE_BASE_PATH, 'conf')

# Crear directorio base si no existe
os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

print(f"🔍 Configurando para usar solo Google Drive...")
print(f"   Ruta base: {DRIVE_BASE_PATH}")
print(f"   Configuración: {drive_conf_dir}")

# Verificar si existe en Google Drive
config_file_drive = os.path.join(drive_conf_dir, 'config.yaml')

if os.path.exists(config_file_drive):
    # Usar configuración de Google Drive
    print(f"✓ Usando configuración de Google Drive: {drive_conf_dir}")
    # Copiar temporalmente al directorio actual para Hydra (requiere ruta relativa)
    import shutil
    current_dir = os.getcwd()
    conf_dir_temp = os.path.join(current_dir, 'conf')

    # Limpiar directorio temporal si existe
    if os.path.exists(conf_dir_temp):
        shutil.rmtree(conf_dir_temp)

    # Copiar configuración de Google Drive al directorio temporal
    shutil.copytree(drive_conf_dir, conf_dir_temp)
    print(f"  ✓ Configuración copiada temporalmente a: {conf_dir_temp}")
    config_path_for_hydra = 'conf'  # Ruta relativa
else:
    # Si no existe, crear los archivos de configuración básicos en Google Drive
    print(f"⚠️ No se encontraron archivos de configuración en Google Drive")
    print(f"   Creando estructura y archivos de configuración básicos en: {drive_conf_dir}")

    # Crear estructura de directorios en Google Drive
    os.makedirs(drive_conf_dir, exist_ok=True)
    os.makedirs(os.path.join(drive_conf_dir, 'model'), exist_ok=True)
    os.makedirs(os.path.join(drive_conf_dir, 'trainer'), exist_ok=True)
    os.makedirs(os.path.join(drive_conf_dir, 'logger'), exist_ok=True)

    # Crear archivo config.yaml básico
    config_yaml_content = f"""# Configuración principal del proyecto
defaults:
  - model: autoencoder_classic
  - trainer: default
  - logger: wandb

# Configuración del dataset
dataset:
  path: "{DRIVE_BASE_PATH}/dataset"
  categories: ["cable", "capsule", "screw", "transistor"]
  selected_category: "cable"
  image_size: 128
  batch_size: 32
  num_workers: 2

# Configuración del modelo
# Los parámetros específicos del modelo están en conf/model/

# Configuración del entrenamiento
trainer:
  max_epochs: 20
  learning_rate: 0.001
  loss_function: "L2"
  scheduler:
    step_size: 10
    gamma: 0.5

# Configuración del logger
logger:
  project: "tarea05-anomaly-detection"
  name: null

# Configuración del dispositivo
device: "cuda"
"""

    with open(config_file_drive, 'w', encoding='utf-8') as f:
        f.write(config_yaml_content)
    print(f"  ✓ Creado: config.yaml")

    # Crear archivo autoencoder_classic.yaml
    # NOTA: Las clases están definidas en el notebook y se registran como notebook_models
    autoencoder_yaml = """# Configuración del autoencoder clásico
# NOTA: _target_ apunta a la clase registrada en notebook_models
_target_: notebook_models.AutoencoderClassic

architecture: "autoencoder_classic"

input_channels: 3
latent_dim: 128

encoder_channels: [64, 128, 256, 512]
decoder_channels: [512, 256, 128, 64]
"""
    with open(os.path.join(drive_conf_dir, 'model', 'autoencoder_classic.yaml'), 'w', encoding='utf-8') as f:
        f.write(autoencoder_yaml)
    print(f"  ✓ Creado: model/autoencoder_classic.yaml")

    # Crear archivo unet.yaml
    unet_yaml = """# Configuración del autoencoder U-net con skip connections
# NOTA: _target_ apunta a la clase registrada en notebook_models
_target_: notebook_models.UNetAutoencoder

architecture: "unet"

input_channels: 3
latent_dim: 128

encoder_channels: [64, 128, 256, 512]
decoder_channels: [512, 256, 128, 64]
"""
    with open(os.path.join(drive_conf_dir, 'model', 'unet.yaml'), 'w', encoding='utf-8') as f:
        f.write(unet_yaml)
    print(f"  ✓ Creado: model/unet.yaml")

    # Crear archivo default.yaml para trainer
    trainer_yaml = """# Configuración del entrenador
max_epochs: 20
learning_rate: 0.001
loss_function: "L2"
scheduler:
  step_size: 10
  gamma: 0.5
"""
    with open(os.path.join(drive_conf_dir, 'trainer', 'default.yaml'), 'w', encoding='utf-8') as f:
        f.write(trainer_yaml)
    print(f"  ✓ Creado: trainer/default.yaml")

    # Crear archivo wandb.yaml
    wandb_yaml = f"""# Configuración de WandB
project: "tarea05-anomaly-detection"
name: null
save_dir: "{DRIVE_BASE_PATH}/wandb_logs"
"""
    with open(os.path.join(drive_conf_dir, 'logger', 'wandb.yaml'), 'w', encoding='utf-8') as f:
        f.write(wandb_yaml)
    print(f"  ✓ Creado: logger/wandb.yaml")

    print(f"✓ Estructura de configuración creada en: {drive_conf_dir}")

    # Copiar temporalmente al directorio actual para Hydra
    import shutil
    current_dir = os.getcwd()
    conf_dir_temp = os.path.join(current_dir, 'conf')
    if os.path.exists(conf_dir_temp):
        shutil.rmtree(conf_dir_temp)
    shutil.copytree(drive_conf_dir, conf_dir_temp)
    print(f"  ✓ Configuración copiada temporalmente a: {conf_dir_temp}")
    config_path_for_hydra = 'conf'  # Ruta relativa

# Verificar que el archivo config.yaml existe antes de inicializar Hydra
# Usar el archivo temporal para Hydra
# IMPORTANTE: Definir current_dir ANTES de usarlo
current_dir = os.getcwd()
conf_dir_temp = os.path.join(current_dir, 'conf')
config_file = os.path.join(conf_dir_temp, 'config.yaml')

if not os.path.exists(config_file):
    raise FileNotFoundError(
        f"❌ ERROR: No se pudo crear o encontrar config.yaml\n"
        f"Se esperaba en: {config_file}\n"
        f"Directorio actual: {current_dir}\n"
        f"Por favor, verifica que los archivos estén en Google Drive: {drive_conf_dir}"
    )

print(f"✓ Archivo config.yaml verificado: {config_file}")

# Limpiar Hydra si ya está inicializado (útil si se ejecuta la celda múltiples veces)
from hydra.core.global_hydra import GlobalHydra
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
    print("✓ Limpiando instancia previa de Hydra")

# IMPORTANTE: Asegurarnos de estar en el directorio correcto ANTES de inicializar Hydra
# Hydra usa el directorio de trabajo actual para buscar la configuración
# Si no estamos en el directorio correcto, Hydra buscará en /tmp
os.chdir(current_dir)
actual_dir = os.getcwd()
print(f"✓ Directorio de trabajo configurado: {actual_dir}")

# Verificar que el directorio conf existe en el directorio actual
if not os.path.exists(conf_dir_temp):
    raise FileNotFoundError(
        f"❌ ERROR: No se encontró el directorio conf en {actual_dir}\n"
        f"Se esperaba: {conf_dir_temp}\n"
        f"Por favor, verifica que los archivos de configuración estén en Google Drive: {drive_conf_dir}"
    )

# Verificar que config.yaml existe
if not os.path.exists(config_file):
    raise FileNotFoundError(
        f"❌ ERROR: No se encontró config.yaml en {conf_dir_temp}\n"
        f"Por favor, verifica que los archivos de configuración estén en Google Drive: {drive_conf_dir}"
    )

print(f"✓ Verificado: {config_file} existe")

# Inicializar Hydra con ruta relativa
# Hydra requiere que config_path sea relativa al directorio de trabajo actual
# IMPORTANTE:
# - config_path debe ser 'conf' (sin barra inicial, sin ruta absoluta)
# - El directorio conf debe estar en el directorio de trabajo actual
# - Hydra buscará en: <directorio_actual>/conf/

# Verificar una vez más que estamos en el directorio correcto
final_dir = os.getcwd()
print(f"✓ Directorio final antes de inicializar Hydra: {final_dir}")
print(f"✓ Verificando que conf existe: {os.path.exists(conf_dir_temp)}")
print(f"✓ Verificando que config.yaml existe: {os.path.exists(config_file)}")

# Inicializar Hydra
# IMPORTANTE: Hydra puede cambiar el directorio de trabajo cuando se inicializa
# Para evitar que busque en /tmp, necesitamos asegurarnos de que:
# 1. El directorio de trabajo actual sea /content (donde está conf/)
# 2. El directorio conf/ esté en el directorio de trabajo actual
# 3. Usar job_name para evitar que Hydra cree directorios temporales

# Verificar que estamos en /content
if os.getcwd() != '/content':
    print(f"⚠️ ADVERTENCIA: El directorio de trabajo es {os.getcwd()}, no /content")
    print(f"   Cambiando a /content...")
    os.chdir('/content')
    print(f"   Nuevo directorio: {os.getcwd()}")

# Verificar nuevamente que conf existe
if not os.path.exists('/content/conf'):
    print(f"❌ ERROR: No se encontró /content/conf")
    print(f"   Copiando desde Google Drive...")
    import shutil
    if os.path.exists(drive_conf_dir):
        if os.path.exists('/content/conf'):
            shutil.rmtree('/content/conf')
        shutil.copytree(drive_conf_dir, '/content/conf')
        print(f"   ✓ Copiado a /content/conf")
    else:
        raise FileNotFoundError(f"No se encontró {drive_conf_dir}")

try:
    print(f"✓ Inicializando Hydra con config_path='{config_path_for_hydra}' desde directorio: {os.getcwd()}")
    # Usar job_name para evitar que Hydra cree directorios temporales
    hydra.initialize(config_path=config_path_for_hydra, version_base=None, job_name="notebook")
    print("✓ Hydra inicializado correctamente")
except Exception as e:
    print(f"❌ Error al inicializar Hydra: {e}")
    print(f"   Directorio actual: {os.getcwd()}")
    print(f"   Conf dir esperado: /content/conf")
    print(f"   Conf dir existe: {os.path.exists('/content/conf')}")
    if os.path.exists('/content/conf'):
        print(f"   Contenido de /content/conf: {os.listdir('/content/conf')}")
    raise

# IMPORTANTE: Registrar las clases del notebook en el resolver de Hydra
# Esto permite que instantiate() encuentre las clases definidas en el notebook
from hydra.core.global_hydra import GlobalHydra
from hydra.core.config_store import ConfigStore

# Crear un módulo temporal para registrar las clases
import sys
import types

# Crear un módulo temporal 'notebook_models' que contenga las clases
notebook_models = types.ModuleType('notebook_models')
notebook_models.AutoencoderClassic = AutoencoderClassic
notebook_models.UNetAutoencoder = UNetAutoencoder
sys.modules['notebook_models'] = notebook_models

# Actualizar los archivos YAML para usar notebook_models
# Actualizar tanto en Google Drive como en el directorio temporal
for yaml_file in [os.path.join(drive_conf_dir, 'model', 'autoencoder_classic.yaml'),
                  os.path.join(drive_conf_dir, 'model', 'unet.yaml'),
                  os.path.join(conf_dir_temp, 'model', 'autoencoder_classic.yaml'),
                  os.path.join(conf_dir_temp, 'model', 'unet.yaml')]:
    if os.path.exists(yaml_file):
        with open(yaml_file, 'r', encoding='utf-8') as f:
            content = f.read()
        # Reemplazar _target_ si apunta a AutoencoderClassic o UNetAutoencoder directamente
        if '_target_: AutoencoderClassic' in content or '_target_: UNetAutoencoder' in content:
            content = content.replace('_target_: AutoencoderClassic', '_target_: notebook_models.AutoencoderClassic')
            content = content.replace('_target_: UNetAutoencoder', '_target_: notebook_models.UNetAutoencoder')
            with open(yaml_file, 'w', encoding='utf-8') as f:
                f.write(content)
            print(f"✓ Actualizado: {os.path.basename(yaml_file)}")

print("✓ Clases del notebook registradas para Hydra")

# Componer la configuración (permite override desde parámetros)

print(os.getcwd())
try:
  print(os.path.exists('/content/conf/config.yaml'))
  os.chdir('/content/conf')
  print(os.getcwd())
  cfg = hydra.compose(config_name="config")
except Exception as e:
  print(f"❌ Error al cargar la configuración: {e}")
  raise
print(cfg)

# Ejemplos de cómo usar overrides para cambiar configuraciones:
#
# 1. Cambiar arquitectura a U-net:
#    cfg = hydra.compose(config_name="config", overrides=["model=unet"])
#
# 2. Cambiar función de pérdida:
#    cfg = hydra.compose(config_name="config", overrides=["trainer.loss_function=SSIM"])
#
# 3. Cambiar learning rate:
#    cfg = hydra.compose(config_name="config", overrides=["trainer.learning_rate=0.0005"])
#
# 4. Cambiar dimensión del espacio latente:
#    cfg = hydra.compose(config_name="config", overrides=["model.latent_dim=256"])
#
# 5. Cambiar categoría del dataset:
#    cfg = hydra.compose(config_name="config", overrides=["dataset.selected_category=capsule"])
#
# 6. Múltiples overrides:
#    cfg = hydra.compose(config_name="config", overrides=["model=unet", "trainer.loss_function=SSIM_L1", "trainer.max_epochs=30"])

print("Configuración cargada con Hydra:")
print(OmegaConf.to_yaml(cfg))

# Extraer valores de configuración
DATASET_PATH = cfg.dataset.path
SELECTED_CATEGORY = cfg.dataset.selected_category
IMAGE_SIZE = cfg.dataset.image_size
BATCH_SIZE = cfg.dataset.batch_size
NUM_WORKERS = cfg.dataset.num_workers

# Configurar dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')

# La variable DRIVE_BASE_PATH está disponible para todo el notebook
# Se usa para guardar modelos, imágenes, resultados, etc. en Google Drive
print(f'✓ Ruta base de Google Drive configurada: {DRIVE_BASE_PATH}')


In [ ]:
# Esta celda está duplicada - el montaje de Google Drive ya se hizo en la celda 2
# Puedes omitir esta celda si ya ejecutaste la celda 2

## 1. Carga y Preprocesamiento de Datos

En esta sección se realiza:
- **Carga del dataset MVTec AD**: Se cargan las rutas de las imágenes de entrenamiento y prueba
- **Definición de transformaciones**: Resize a 128x128, normalización y conversión a tensores
- **Creación de DataLoaders**: Preparación de los datos para el entrenamiento con división train/validation (80-20)

### Nota Importante sobre Normalización

**Corrección aplicada**: Las transformaciones utilizan normalización a **[-1, 1]** en lugar de las estadísticas de ImageNet. Esto es necesario porque:
- El decoder del modelo utiliza `Tanh()` como función de activación final, que produce valores en el rango **[-1, 1]**
- Si las imágenes de entrada están normalizadas con estadísticas de ImageNet (rango aproximado [-2.5, 2.5]), hay una incompatibilidad de rangos
- La normalización a **[-1, 1]** se logra con `Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])` después de `ToTensor()` que convierte a [0, 1]
- Esta corrección permite que el modelo reconstruya correctamente las imágenes


### 1.1. Definición del Dataset y Carga de Rutas
La siguiente celda:
- Define la clase `AnomalyDataset` para cargar imágenes del dataset MVTec AD
- Define la función `load_dataset_paths` para obtener las rutas de las imágenes
- Carga las rutas de entrenamiento (solo imágenes 'good') y prueba (todas las clases)
- Muestra el número de imágenes en cada conjunto

In [ ]:
class AnomalyDataset(Dataset):
    """Dataset para cargar imágenes de entrenamiento y prueba"""

    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

def load_dataset_paths(category_path, split='train'):
    """Carga las rutas de las imágenes del dataset"""
    paths = []
    split_path = os.path.join(category_path, split)

    if split == 'train':
        # Solo imágenes 'good' en entrenamiento
        good_path = os.path.join(split_path, 'good')
        if os.path.exists(good_path):
            for img_file in os.listdir(good_path):
                if img_file.endswith('.png'):
                    paths.append(os.path.join(good_path, img_file))
    else:
        # En test, cargar todas las clases (good y defectos)
        if os.path.exists(split_path):
            for class_name in os.listdir(split_path):
                class_path = os.path.join(split_path, class_name)
                if os.path.isdir(class_path):
                    for img_file in os.listdir(class_path):
                        if img_file.endswith('.png'):
                            paths.append(os.path.join(class_path, img_file))

    return paths

# Cargar rutas del dataset
category_path = os.path.join(DATASET_PATH, SELECTED_CATEGORY)
train_paths = load_dataset_paths(category_path, split='train')
test_paths = load_dataset_paths(category_path, split='test')

print(f'Imágenes de entrenamiento: {len(train_paths)}')
print(f'Imágenes de prueba: {len(test_paths)}')


Esta celda:
- Define las transformaciones de datos (resize, normalización a [-1, 1])
- Crea los datasets de entrenamiento y prueba
- Divide el set de entrenamiento en train/validation (80-20)
- Crea los DataLoaders para cada conjunto

**Nota sobre normalización**: Se usa normalización a [-1, 1] para compatibilidad con `Tanh()` en el decoder.

In [ ]:
# Transformaciones de datos (usando configuración de Hydra)
# IMPORTANTE: Normalización a [-1, 1] para compatibilidad con Tanh() en el decoder
# ToTensor() convierte a [0, 1], luego normalizamos a [-1, 1] con (x - 0.5) / 0.5
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza a [-1, 1]
])

test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza a [-1, 1]
])

# Crear datasets
train_dataset = AnomalyDataset(train_paths, transform=train_transform)
test_dataset = AnomalyDataset(test_paths, transform=test_transform)

# Crear dataloaders (usando configuración de Hydra)
# Dividir el set de entrenamiento en train y validation (80-20) para WandB
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset_split, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset_split, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print(f'Batches de entrenamiento: {len(train_loader)}')
print(f'Batches de validación: {len(val_loader)}')
print(f'Batches de prueba: {len(test_loader)}')


## 2. Entrenamiento de Modelos

Esta sección contiene dos opciones para entrenar los modelos:

### 2.1. Ejecución Automatizada de Experimentos (Recomendado)

Esta opción ejecuta automáticamente **todos los experimentos requeridos**:
- **2 arquitecturas**: Autoencoder Clásico y U-net
- **4 funciones de pérdida**: L1, L2, SSIM, SSIM_L1
- **Total: 8 experimentos**

**Proceso automatizado**:
1. Itera sobre todas las combinaciones de arquitectura y función de pérdida
2. Entrena cada modelo usando Pytorch Lightning con callbacks de visualización
3. Evalúa el modelo y calcula métricas (AUC-ROC, Average Precision)
4. Registra todo en WandB
5. Genera un resumen comparativo al final

** IMPORTANTE**: Esta celda debe ejecutarse DESPUÉS de haber definido las funciones `AnomalyDataset`, `load_dataset_paths` y `get_anomaly_labels` en las celdas anteriores.


### 2.2. Experimento Individual (Opcional)

Esta sección permite ejecutar **un solo experimento** de forma manual, útil para:
- Probar configuraciones específicas
- Depurar problemas
- Realizar experimentos personalizados

**Pasos**:
1. Instanciar el modelo usando Hydra (permite cambiar arquitectura con overrides)
2. Crear el módulo Lightning con la función de pérdida especificada
3. Configurar WandB logger y callbacks
4. Entrenar el modelo

**Nota**: Si ejecutaste la celda de automatización (2.1), puedes saltar esta sección.


In [ ]:
# Ejecución automatizada de todos los experimentos
import pandas as pd
from datetime import datetime
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from hydra.utils import instantiate

# Definir función auxiliar para obtener etiquetas de anomalía
def get_anomaly_labels(test_paths, category_path):
    """Obtiene las etiquetas de anomalía (0=normal, 1=anomalía)"""
    labels = []
    good_path = os.path.join(category_path, 'test', 'good')

    for path in test_paths:
        if good_path in path:
            labels.append(0)  # Normal
        else:
            labels.append(1)  # Anomalía

    return np.array(labels)

# Configuración de experimentos
ARCHITECTURES = ["autoencoder_classic", "unet"]
LOSS_FUNCTIONS = ["L1", "L2", "SSIM", "SSIM_L1"]

# Almacenar resultados de todos los experimentos
experiment_results = []

print("=" * 80)
print("INICIANDO EJECUCIÓN AUTOMATIZADA DE EXPERIMENTOS")
print("=" * 80)
print(f"Total de experimentos: {len(ARCHITECTURES) * len(LOSS_FUNCTIONS)}")
print(f"Arquitecturas: {ARCHITECTURES}")
print(f"Funciones de pérdida: {LOSS_FUNCTIONS}")
print("=" * 80)

# Iterar sobre todas las combinaciones
for arch_idx, architecture in enumerate(ARCHITECTURES):
    for loss_idx, loss_function in enumerate(LOSS_FUNCTIONS):
        exp_num = arch_idx * len(LOSS_FUNCTIONS) + loss_idx + 1
        total_exps = len(ARCHITECTURES) * len(LOSS_FUNCTIONS)

        print(f"\n{'='*80}")
        print(f"EXPERIMENTO {exp_num}/{total_exps}")
        print(f"Arquitectura: {architecture}")
        print(f"Función de pérdida: {loss_function}")
        print(f"{'='*80}\n")

        # Componer configuración con overrides
        cfg = hydra.compose(
            config_name="config",
            overrides=[
                f"model={architecture}",
                f"trainer.loss_function={loss_function}"
            ]
        )

        # Extraer valores de configuración
        DATASET_PATH = cfg.dataset.path
        SELECTED_CATEGORY = cfg.dataset.selected_category
        IMAGE_SIZE = cfg.dataset.image_size
        BATCH_SIZE = cfg.dataset.batch_size
        NUM_WORKERS = cfg.dataset.num_workers

        # Cargar rutas del dataset
        category_path = os.path.join(DATASET_PATH, SELECTED_CATEGORY)
        train_paths = load_dataset_paths(category_path, split='train')
        test_paths = load_dataset_paths(category_path, split='test')

        # Crear transformaciones
        # IMPORTANTE: Normalización a [-1, 1] para compatibilidad con Tanh() en el decoder
        train_transform = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza a [-1, 1]
        ])

        test_transform = transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normaliza a [-1, 1]
        ])

        # Crear datasets y dataloaders
        train_dataset = AnomalyDataset(train_paths, transform=train_transform)
        test_dataset = AnomalyDataset(test_paths, transform=test_transform)

        # Dividir train en train y validation (80-20)
        from torch.utils.data import random_split
        train_size = int(0.8 * len(train_dataset))
        val_size = len(train_dataset) - train_size
        train_dataset_split, val_dataset = random_split(train_dataset, [train_size, val_size])

        train_loader = DataLoader(train_dataset_split, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

        # Instanciar modelo
        base_model = instantiate(cfg.model)
        base_model = base_model.to(device)

        # Crear módulo Lightning
        lightning_model = AutoencoderLightning(
            model=base_model,
            learning_rate=cfg.trainer.learning_rate,
            loss_function=cfg.trainer.loss_function,
            scheduler_config=cfg.trainer.scheduler
        )

        # Configurar logger de WandB para este experimento
        wandb_logger = WandbLogger(
            project=cfg.logger.project,
            name=f"{architecture}-{loss_function}-{SELECTED_CATEGORY}",
            config=OmegaConf.to_container(cfg, resolve=True),
            reinit=True
        )

        # Callbacks
        checkpoint_dir = os.path.join(DRIVE_BASE_PATH, 'checkpoints', f'{architecture}-{loss_function}')
        os.makedirs(checkpoint_dir, exist_ok=True)
        checkpoint_callback = ModelCheckpoint(
            monitor='train/loss_epoch',
            dirpath=checkpoint_dir,
            filename=f'{architecture}-{loss_function}-{{epoch:02d}}-{{train/loss_epoch:.4f}}',
            save_top_k=1,
            mode='min'
        )

        lr_monitor = LearningRateMonitor(logging_interval='step')

        # Callback de visualización periódica (cada 5 épocas)
        viz_callback = VisualizationCallback(
            val_loader=val_loader,
            device=device,
            log_every_n_epochs=5,
            num_samples=16
        )

        # Crear Trainer
        trainer = Trainer(
            max_epochs=cfg.trainer.max_epochs,
            accelerator='auto',
            devices=1,
            logger=wandb_logger,
            callbacks=[checkpoint_callback, lr_monitor, viz_callback],
            log_every_n_steps=10,
            enable_progress_bar=True
        )

        # Entrenar con validación
        print(f"\nIniciando entrenamiento...")
        trainer.fit(lightning_model, train_loader, val_loader)

        # Obtener pérdidas de entrenamiento y validación
        train_losses = lightning_model.train_losses if hasattr(lightning_model, 'train_losses') else []

        # Obtener pérdidas de validación del trainer
        val_losses = []
        if hasattr(trainer, 'callback_metrics') and 'val/loss_epoch' in trainer.callback_metrics:
            # Si hay una pérdida de validación en las métricas del callback
            val_loss_epoch = trainer.callback_metrics.get('val/loss_epoch')
            if val_loss_epoch is not None:
                # Si solo hay un valor, crear una lista con ese valor repetido para cada época
                if len(train_losses) > 0:
                    val_losses = [val_loss_epoch.item() if hasattr(val_loss_epoch, 'item') else val_loss_epoch] * len(train_losses)

        # Si no se pudieron obtener las pérdidas de validación, intentar obtenerlas del historial del trainer
        if len(val_losses) == 0 and hasattr(trainer, 'logged_metrics'):
            # Intentar obtener del historial de métricas
            logged_metrics = trainer.logged_metrics
            if 'val/loss' in logged_metrics:
                val_loss_val = logged_metrics['val/loss']
                if hasattr(val_loss_val, 'item'):
                    val_losses = [val_loss_val.item()] * len(train_losses) if len(train_losses) > 0 else []

        final_train_loss = train_losses[-1] if train_losses else None
        final_val_loss = val_losses[-1] if val_losses and len(val_losses) > 0 else None

        # Evaluar
        print(f"\nEvaluando modelo...")
        lightning_model.model.eval()
        anomaly_scores = []

        with torch.no_grad():
            for images in test_loader:
                images = images.to(device)
                reconstructed = lightning_model.model(images)
                mse = torch.mean((images - reconstructed) ** 2, dim=(1, 2, 3))
                anomaly_scores.extend(mse.cpu().numpy())

        anomaly_scores = np.array(anomaly_scores)

        # Obtener etiquetas
        test_labels = get_anomaly_labels(test_paths, category_path)

        # Calcular métricas
        auc_score = roc_auc_score(test_labels, anomaly_scores)
        ap_score = average_precision_score(test_labels, anomaly_scores)

        # Guardar resultados
        result = {
            'experiment': exp_num,
            'architecture': architecture,
            'loss_function': loss_function,
            'category': SELECTED_CATEGORY,
            'final_train_loss': final_train_loss,
            'final_val_loss': final_val_loss,
            'train_losses': train_losses,
            'val_losses': val_losses,
            'auc_roc': auc_score,
            'average_precision': ap_score,
            'min_score': float(anomaly_scores.min()),
            'max_score': float(anomaly_scores.max()),
            'mean_score': float(anomaly_scores.mean()),
            'std_score': float(anomaly_scores.std())
        }
        experiment_results.append(result)

        # Loggear métricas finales en wandb usando el logger del experimento
        # NOTA: Usamos wandb_logger.experiment.log() en lugar de wandb.log() para mantener consistencia
        # El WandbLogger finaliza automáticamente el run cuando el Trainer termina
        try:
            if wandb_logger is not None:
                wandb_logger.experiment.log({
                    "eval/auc_roc": auc_score,
                    "eval/average_precision": ap_score,
                    "eval/final_train_loss": final_train_loss if final_train_loss else 0.0,
                    "eval/final_val_loss": final_val_loss if final_val_loss else 0.0
                })
        except Exception as e:
            print(f"⚠️ Error al loggear métricas finales en wandb: {e}")

        # NOTA: No finalizamos WandB aquí porque:
        # 1. El WandbLogger de PyTorch Lightning finaliza automáticamente el run cuando el Trainer termina
        # 2. Cada experimento tiene su propio run con WandbLogger(reinit=True), así que se finaliza automáticamente
        # 3. Si necesitamos finalizar manualmente, se hará al final de todos los experimentos (ver celda 6.2)
        # wandb.finish()  # Comentado: el logger finaliza automáticamente

        print(f"\n✓ Experimento {exp_num}/{total_exps} completado")
        print(f"  AUC-ROC: {auc_score:.4f}")
        print(f"  Average Precision: {ap_score:.4f}")
        train_loss_str = f"{final_train_loss:.4f}" if final_train_loss is not None else "N/A"
        val_loss_str = f"{final_val_loss:.4f}" if final_val_loss is not None else "N/A"
        print(f"  Pérdida final train: {train_loss_str}")
        print(f"  Pérdida final val: {val_loss_str}")

        # Limpiar memoria
        del lightning_model
        del base_model
        del trainer
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# Crear DataFrame con resultados
results_df = pd.DataFrame(experiment_results)

# Guardar resultados
results_path = os.path.join(DRIVE_BASE_PATH, 'resultados_experimentos.csv')
results_df.to_csv(results_path, index=False)

print(f"\n{'='*80}")
print("TODOS LOS EXPERIMENTOS COMPLETADOS")
print(f"{'='*80}")
print(f"\nResultados guardados en: {results_path}")
print("\nResumen de resultados:")
print(results_df.to_string(index=False))

# Crear tabla comparativa
print(f"\n{'='*80}")
print("TABLA COMPARATIVA DE RESULTADOS")
print(f"{'='*80}")
comparison = results_df.pivot_table(
    index='architecture',
    columns='loss_function',
    values=['auc_roc', 'average_precision'],
    aggfunc='first'
)
print(comparison)

# Guardar tabla comparativa
comparison_path = os.path.join(DRIVE_BASE_PATH, 'comparacion_experimentos.csv')
comparison.to_csv(comparison_path)
print(f"\nTabla comparativa guardada en: {comparison_path}")

# Visualizar comparación
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de AUC-ROC
auc_pivot = results_df.pivot_table(
    index='architecture',
    columns='loss_function',
    values='auc_roc',
    aggfunc='first'
)
auc_pivot.plot(kind='bar', ax=axes[0], rot=0)
axes[0].set_title('Comparación de AUC-ROC por Arquitectura y Función de Pérdida')
axes[0].set_ylabel('AUC-ROC')
axes[0].legend(title='Función de Pérdida')
axes[0].grid(True, alpha=0.3)

# Gráfico de Average Precision
ap_pivot = results_df.pivot_table(
    index='architecture',
    columns='loss_function',
    values='average_precision',
    aggfunc='first'
)
ap_pivot.plot(kind='bar', ax=axes[1], rot=0)
axes[1].set_title('Comparación de Average Precision por Arquitectura y Función de Pérdida')
axes[1].set_ylabel('Average Precision')
axes[1].legend(title='Función de Pérdida')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_path = os.path.join(DRIVE_BASE_PATH, 'comparacion_grafica.png')
plt.savefig(comparison_plot_path)
plt.show()

print(f"\n{'='*80}")
print("ANÁLISIS COMPLETO")
print(f"{'='*80}")
print("\nMejor resultado por métrica:")
        best_auc = results_df.loc[results_df['auc_roc'].idxmax()]
        print(f"Mejor AUC-ROC: Arquitectura={best_auc['architecture']}, Loss={best_auc['loss_function']}, AUC={best_auc['auc_roc']:.4f}")
        best_ap = results_df.loc[results_df['average_precision'].idxmax()]
        print(f"Mejor Average Precision: Arquitectura={best_ap['architecture']}, Loss={best_ap['loss_function']}, AP={best_ap['average_precision']:.4f}")

        # Finalizar cualquier run de WandB que quede activo después de todos los experimentos
        # NOTA: Cada experimento tiene su propio run que se finaliza automáticamente cuando el Trainer termina
        # Sin embargo, si hay algún run activo, lo finalizamos aquí
        try:
            if 'wandb' in globals() and wandb.run is not None:
                if not wandb.run.settings._disable:
                    wandb.finish()
                    print("\n✓ Todos los runs de WandB finalizados")
        except Exception as e:
            if "finished" not in str(e).lower() and "is finished" not in str(e).lower():
                print(f"⚠️ Error al finalizar WandB: {e}")
            # Si ya está finalizado, no hay problema

### 2.2.1. Instanciación del Modelo

En esta celda se instancia el modelo usando Hydra. El modelo se crea automáticamente según la configuración definida en `conf/model/`.

**Uso de overrides**:
- Para cambiar a U-net: `cfg = hydra.compose(config_name="config", overrides=["model=unet"])`
- Para cambiar función de pérdida: `cfg = hydra.compose(config_name="config", overrides=["trainer.loss_function=SSIM"])`

Hydra instanciará automáticamente el modelo según la configuración en `conf/model/`.

In [ ]:

from hydra.utils import instantiate

# Instanciar el modelo automáticamente desde la configuración
# Esto usa el _target_ y parámetros definidos en conf/model/autoencoder_classic.yaml o conf/model/unet.yaml
base_model = instantiate(cfg.model)

print(f"Modelo instanciado: {cfg.model._target_}")
print(f"Parámetros del modelo: {sum(p.numel() for p in base_model.parameters())}")

# Crear módulo Lightning con la función de pérdida especificada
lightning_model = AutoencoderLightning(
    model=base_model,
    learning_rate=cfg.trainer.learning_rate,
    loss_function=cfg.trainer.loss_function,
    scheduler_config=cfg.trainer.scheduler
)

print(f"\nArquitectura: {cfg.model.architecture}")
print(f"Función de pérdida: {cfg.trainer.loss_function}")
print(f"Learning rate: {cfg.trainer.learning_rate}")
print(f"Latent dim: {cfg.model.latent_dim}")
print(f"Input channels: {cfg.model.input_channels}")


## 3. Entrenamiento del Modelo

En esta sección se entrena el modelo usando Pytorch Lightning.

**Características del entrenamiento**:
- **Callbacks automáticos**:
  - `ModelCheckpoint`: Guarda el mejor modelo según la pérdida de entrenamiento
  - `LearningRateMonitor`: Monitorea el learning rate durante el entrenamiento
  - `VisualizationCallback`: Genera visualizaciones periódicas cada 5 épocas (reconstrucciones y t-SNE del espacio latente)
- **Logging en WandB**: Todas las métricas se registran automáticamente en Weights & Biases
- **Visualizaciones periódicas**: Durante el entrenamiento, cada 5 épocas se generan y registran en WandB:
  - Comparación de reconstrucciones del set de validación (16 imágenes)
  - Visualización t-SNE del espacio latente del set de validación


### 3.1. Entrenamiento del Modelo (Experimento Individual)

Esta celda entrena el modelo usando Pytorch Lightning para un experimento individual.

**Nota**: Si ejecutaste la celda de automatización (2.1), puedes saltar esta sección ya que el entrenamiento ya se realizó automáticamente.

### 3.3.1. Preparación para Visualizaciones

Esta celda define la función auxiliar `denormalize` necesaria para visualizar las imágenes correctamente.

Las visualizaciones del set de validación se generan automáticamente durante el entrenamiento mediante el `VisualizationCallback` (cada 5 épocas).

Si necesitas generar visualizaciones adicionales después del entrenamiento, puedes usar las celdas siguientes.

In [ ]:
# Función auxiliar para denormalizar imágenes
def denormalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    """Desnormaliza un tensor de imagen de [-1, 1] a [0, 1] para visualización"""
    # Desnormalizar: x = (x_norm * std) + mean
    # Para normalización [-1, 1]: x_norm = (x - 0.5) / 0.5, entonces x = (x_norm * 0.5) + 0.5
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor.clamp_(0, 1)

print("✓ Función denormalize definida correctamente")


In [ ]:
# Visualización de reconstrucciones del set de validación (16 imágenes)
lightning_model.model.eval()
val_images = []
val_reconstructions = []
val_latent_vectors = []

with torch.no_grad():
    for batch_idx, images in enumerate(val_loader):
        images = images.to(device)

        # Obtener reconstrucciones
        reconstructed = lightning_model.model(images)

        # Extraer vectores latentes
        if hasattr(lightning_model.model, 'encode'):
            latent = lightning_model.model.encode(images)
            # Para UNet, encode devuelve (latent, skip_connections), solo necesitamos latent
            if isinstance(latent, tuple):
                latent = latent[0]
            # Aplanar el vector latente para t-SNE
            latent_flat = latent.view(latent.size(0), -1)
            val_latent_vectors.append(latent_flat.cpu().numpy())

        # Guardar imágenes para visualización (hasta 16)
        if len(val_images) < 16:
            remaining = 16 - len(val_images)
            val_images.append(images[:remaining].cpu())
            val_reconstructions.append(reconstructed[:remaining].cpu())

        if len(val_images) >= 16:
            break

# Concatenar todas las imágenes
val_images = torch.cat(val_images[:16])[:16]
val_reconstructions = torch.cat(val_reconstructions[:16])[:16]

# Visualizar 16 reconstrucciones del set de validación
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
axes = axes.flatten()
val_reconstruction_images = []

for i in range(16):
    # Imagen original
    img = denormalize(val_images[i].clone())
    axes[i*2].imshow(img.permute(1, 2, 0))
    axes[i*2].set_title(f'Original {i+1}')
    axes[i*2].axis('off')

    # Reconstrucción
    recon = denormalize(val_reconstructions[i].clone())
    axes[i*2+1].imshow(recon.permute(1, 2, 0))
    axes[i*2+1].set_title(f'Reconstrucción {i+1}')
    axes[i*2+1].axis('off')

    # Preparar para wandb
    img_np = (img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    recon_np = (recon.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    val_reconstruction_images.append(wandb.Image(img_np, caption=f"Original Val {i+1}"))
    val_reconstruction_images.append(wandb.Image(recon_np, caption=f"Reconstrucción Val {i+1}"))

plt.tight_layout()
validation_recon_path = os.path.join(DRIVE_BASE_PATH, 'validation_reconstructions.png')
plt.savefig(validation_recon_path)
plt.show()

# Loggear en wandb (usar wandb_logger si está disponible y el run está activo)
# Profe Steven ve esto, deje estas condicionales para hacer pruebas con la sesion cerrada y abierta de w&b
try:
    if 'wandb_logger' in globals() and wandb_logger is not None:
        # Verificar si el experimento está activo
        if hasattr(wandb_logger.experiment, 'id') and wandb_logger.experiment.id is not None:
            try:
                wandb_logger.experiment.log({
                    "val/reconstructions": val_reconstruction_images,
                    "val/reconstructions_grid": wandb.Image(validation_recon_path)
                })
                print("✓ Reconstrucciones del set de validación guardadas en wandb")
            except Exception as e:
                # Si el run está finalizado, solo guardar localmente
                if "finished" in str(e).lower() or "is finished" in str(e).lower():
                    print("ℹ️ El run de WandB ya está finalizado. Las visualizaciones se guardaron localmente.")
                    print(f"   Ruta: {validation_recon_path}")
                    print("   Nota: Estas visualizaciones ya se generaron automáticamente durante el entrenamiento.")
                else:
                    raise
        else:
            print(f"ℹ️ Reconstrucciones guardadas en: {validation_recon_path}")
            print("   Nota: El run de WandB no está activo. Las visualizaciones ya se generaron durante el entrenamiento.")
    elif 'wandb' in globals() and wandb.run is not None and not wandb.run.settings._disable:
        # Si wandb está inicializado directamente y el run está activo
        try:
            wandb.log({
                "val/reconstructions": val_reconstruction_images,
                "val/reconstructions_grid": wandb.Image(validation_recon_path)
            })
            print("✓ Reconstrucciones del set de validación guardadas en wandb")
        except Exception as e:
            if "finished" in str(e).lower() or "is finished" in str(e).lower():
                print("ℹ️ El run de WandB ya está finalizado. Las visualizaciones se guardaron localmente.")
                print(f"   Ruta: {validation_recon_path}")
            else:
                raise
    else:
        print(f"ℹ️ Reconstrucciones guardadas en: {validation_recon_path}")
        print("   Nota: wandb_logger no está definido o el run está finalizado.")
        print("   Las visualizaciones ya se generaron automáticamente durante el entrenamiento mediante el VisualizationCallback.")
except Exception as e:
    # Manejar cualquier otro error silenciosamente
    print(f"ℹ️ Reconstrucciones guardadas en: {validation_recon_path}")
    if "finished" not in str(e).lower() and "is finished" not in str(e).lower():
        print(f"   (Error menor al loggear en wandb: {e})")
    print("   Nota: Las visualizaciones ya se generaron automáticamente durante el entrenamiento.")


### 3.3.2. Reconstrucciones del Set de Validación

Esta celda genera visualizaciones de reconstrucciones del set de validación (16 imágenes) **después del entrenamiento**.

**Nota**: Estas visualizaciones también se generan automáticamente durante el entrenamiento cada 5 épocas mediante el `VisualizationCallback`.

### 3.3.3. Visualización t-SNE del Espacio Latente

Esta celda genera la visualización t-SNE del espacio latente del set de validación **después del entrenamiento**.

**Nota**: Esta visualización también se genera automáticamente durante el entrenamiento cada 5 épocas mediante el `VisualizationCallback`.


In [ ]:
from sklearn.manifold import TSNE

# Verificar si val_latent_vectors existe, si no, extraerlos
if 'val_latent_vectors' not in globals() or not val_latent_vectors:
    print("Extrayendo vectores latentes del set de validación...")
    val_latent_vectors = []

    # Verificar que el modelo y val_loader estén disponibles
    if 'lightning_model' in globals() and 'val_loader' in globals():
        lightning_model.model.eval()
        with torch.no_grad():
            for batch_idx, images in enumerate(val_loader):
                images = images.to(device)

                # Extraer vectores latentes
                if hasattr(lightning_model.model, 'encode'):
                    latent = lightning_model.model.encode(images)
                    # Para UNet, encode devuelve (latent, skip_connections), solo necesitamos latent
                    if isinstance(latent, tuple):
                        latent = latent[0]
                    # Aplanar el vector latente para t-SNE
                    latent_flat = latent.view(latent.size(0), -1)
                    val_latent_vectors.append(latent_flat.cpu().numpy())
        print(f"✓ Vectores latentes extraídos: {len(val_latent_vectors)} batches")
    else:
        print("Error: lightning_model o val_loader no están disponibles.")
        print("Por favor, ejecuta primero las celdas de entrenamiento y carga de datos.")

# Concatenar todos los vectores latentes
if val_latent_vectors:
    all_latent_vectors = np.concatenate(val_latent_vectors, axis=0)

    print(f"Forma de los vectores latentes: {all_latent_vectors.shape}")
    print("Aplicando t-SNE...")

    # Aplicar t-SNE (reducir a 2D para visualización)
    # Asegurarse de que perplexity sea menor que el número de muestras
    perplexity_val = min(30, len(all_latent_vectors) - 1)
    if perplexity_val <= 0: # Handle case with very few samples
        print("No hay suficientes muestras para t-SNE con perplexity > 0.")
    else:
        tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_val)
        latent_2d = tsne.fit_transform(all_latent_vectors)

        # Visualizar t-SNE
        plt.figure(figsize=(10, 8))
        plt.scatter(latent_2d[:, 0], latent_2d[:, 1], alpha=0.6, s=50)
        plt.title('t-SNE del Espacio Latente (Set de Validación)')
        plt.xlabel('Componente t-SNE 1')
        plt.ylabel('Componente t-SNE 2')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        tsne_path = os.path.join(DRIVE_BASE_PATH, 'tsne_latent_validation.png')
        plt.savefig(tsne_path)
        plt.show()

        # Loggear en wandb (verificar si wandb_logger está definido)
        try:
            if 'wandb_logger' in globals() and wandb_logger is not None:
                wandb_logger.experiment.log({
                    "val/tsne_latent": wandb.Image(tsne_path)
                })
                print("t-SNE del espacio latente guardado en wandb")
            elif 'wandb' in globals() and wandb.run is not None:
                # Verificar que wandb está inicializado
                wandb.log({
                    "val/tsne_latent": wandb.Image(tsne_path)
                })
                print("t-SNE del espacio latente guardado en wandb")
            else:
                print("t-SNE del espacio latente guardado en:", tsne_path)
                print("Nota: wandb_logger no está definido. Ejecuta primero la celda de entrenamiento para habilitar el logging.")
        except Exception as e:
            print(f"Error al loggear en wandb: {e}")
            print("t-SNE del espacio latente guardado en:", tsne_path)
else:
    print("No se pudieron extraer vectores latentes. Verifica que el modelo tenga método encode().")

## 3.3. Visualizaciones del Set de Validación (Requisitos WandB)

Esta sección genera las visualizaciones requeridas para WandB del set de validación **después del entrenamiento**:

1. **Reconstrucciones del set de validación (16 imágenes)**: Comparación lado a lado de imágenes originales vs reconstruidas
2. **t-SNE del espacio latente**: Visualización 2D de cómo el modelo organiza las imágenes en el espacio latente

**Nota**:
- Estas visualizaciones también se generan **automáticamente durante el entrenamiento** cada 5 épocas mediante el `VisualizationCallback`
- Las visualizaciones se registran automáticamente en WandB con el nombre del experimento correspondiente


In [ ]:
# Entrenar modelo usando Pytorch Lightning Trainer
# Verificar e instalar dependencias si es necesario
try:
    import pytorch_lightning as pl
    from pytorch_lightning import Trainer
    from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
except ImportError:
    print("pytorch-lightning no encontrado. Instalando...")
    try:
        # Intentar usar %pip (compatible con Colab)
        get_ipython().run_line_magic('pip', 'install pytorch-lightning')
    except:
        # Si no funciona, usar subprocess
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "pytorch-lightning"])
    from pytorch_lightning import Trainer
    from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
    print("✓ pytorch-lightning instalado correctamente")

# Configurar logger de WandB para Lightning
wandb_logger = WandbLogger(
    project=cfg.logger.project,
    name=f"{cfg.model.architecture}-{cfg.trainer.loss_function}-{SELECTED_CATEGORY}",
    config=OmegaConf.to_container(cfg, resolve=True)
)

# Callbacks
checkpoint_dir = os.path.join(DRIVE_BASE_PATH, 'checkpoints')
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_callback = ModelCheckpoint(
    monitor='train/loss_epoch',
    dirpath=checkpoint_dir,
    filename=f'{cfg.model.architecture}-{cfg.trainer.loss_function}-{{epoch:02d}}-{{train/loss_epoch:.4f}}',
    save_top_k=1,
    mode='min'
)
lr_monitor = LearningRateMonitor(logging_interval='step')

# Callback de visualización periódica (cada 5 épocas)
# Genera visualizaciones de reconstrucciones y t-SNE durante el entrenamiento
viz_callback = VisualizationCallback(
    val_loader=val_loader,
    device=device,
    log_every_n_epochs=5,
    num_samples=16
)

# Crear Trainer de Pytorch Lightning
trainer = Trainer(
    max_epochs=cfg.trainer.max_epochs,
    accelerator='auto',
    devices=1,
    logger=wandb_logger,
    callbacks=[checkpoint_callback, lr_monitor, viz_callback],
    log_every_n_steps=10,
    enable_progress_bar=True
)

# Entrenar modelo
print(f'Iniciando entrenamiento con Pytorch Lightning...')
print(f'Arquitectura: {cfg.model.architecture}')
print(f'Función de pérdida: {cfg.trainer.loss_function}')
print(f'Learning rate: {cfg.trainer.learning_rate}')
trainer.fit(lightning_model, train_loader, val_loader) # Añadir val_loader para que el logger monitoree 'val/loss'
print('\nEntrenamiento completado!')

# Obtener pérdidas de entrenamiento del modelo Lightning
train_losses = lightning_model.train_losses if hasattr(lightning_model, 'train_losses') else []

# Obtener el modelo base entrenado para evaluación
model = lightning_model.model

### 3.2. Visualización de Curva de Pérdida

Esta celda genera la gráfica de la curva de pérdida de entrenamiento a lo largo de las épocas.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses)
plt.title('Curva de Pérdida de Entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida (MSE)')
plt.grid(True)
train_loss_path = os.path.join(DRIVE_BASE_PATH, 'train_loss_curve.png')
plt.savefig(train_loss_path)
plt.show()

# Loggear la curva de pérdida en wandb
wandb_logger.experiment.log({"train/loss_curve": wandb.Image(train_loss_path)})

## 4. Evaluación y Detección de Anomalías

En esta sección se evalúa el modelo entrenado en el set de prueba:

1. **Cálculo de scores de anomalía**: Se usa el error de reconstrucción (MSE) como métrica
2. **Selección de imágenes**: Se seleccionan 8 imágenes buenas y 8 con anomalías para visualización
3. **Cálculo de métricas**: AUC-ROC y Average Precision
4. **Visualización de curva ROC**: Gráfico de la curva ROC para evaluar el rendimiento

**Métricas calculadas**:
- **AUC-ROC**: Área bajo la curva ROC (mientras más cercano a 1.0, mejor)
- **Average Precision**: Precisión promedio (mientras más cercano a 1.0, mejor)


### 4.1. Obtención de Etiquetas de Anomalía

Esta celda define la función para obtener las etiquetas de anomalía (0=normal, 1=anomalía) del set de prueba y muestra la distribución de clases.

In [ ]:
def get_anomaly_labels(test_paths, category_path):
    """Obtiene las etiquetas de anomalía (0=normal, 1=anomalía)"""
    labels = []
    good_path = os.path.join(category_path, 'test', 'good')

    for path in test_paths:
        if good_path in path:
            labels.append(0)  # Normal
        else:
            labels.append(1)  # Anomalía

    return np.array(labels)

# Obtener etiquetas de prueba
test_labels = get_anomaly_labels(test_paths, category_path)
print(f'Imágenes normales: {np.sum(test_labels == 0)}')
print(f'Imágenes con anomalías: {np.sum(test_labels == 1)}')


### 4.2. Evaluación del Modelo en Set de Prueba

Esta celda evalúa el modelo entrenado en el set de prueba:
- Calcula scores de anomalía usando el error de reconstrucción (MSE)
- Selecciona 8 imágenes buenas y 8 con anomalías para visualización
- Prepara los datos para el cálculo de métricas

In [ ]:
lightning_model.model.eval()
anomaly_scores = []
test_images_all = []
test_reconstructions_all = []
test_paths_all = []

with torch.no_grad():
    for batch_idx, images in enumerate(test_loader):
        images = images.to(device)
        reconstructed = lightning_model.model(images)

        # Calcular error de reconstrucción (MSE) como score de anomalía
        mse = torch.mean((images - reconstructed) ** 2, dim=(1, 2, 3))
        anomaly_scores.extend(mse.cpu().numpy())

        # Guardar todas las imágenes y reconstrucciones con sus rutas
        batch_start = batch_idx * BATCH_SIZE
        batch_paths = test_paths[batch_start:batch_start + len(images)]
        test_images_all.append(images.cpu())
        test_reconstructions_all.append(reconstructed.cpu())
        test_paths_all.extend(batch_paths)

anomaly_scores = np.array(anomaly_scores)
print(f'Rango de scores: [{anomaly_scores.min():.4f}, {anomaly_scores.max():.4f}]')

# Obtener etiquetas
category_path = os.path.join(DATASET_PATH, SELECTED_CATEGORY)
test_labels = get_anomaly_labels(test_paths_all, category_path)

# Separar imágenes buenas y con anomalías
good_indices = np.where(test_labels == 0)[0]
anomaly_indices = np.where(test_labels == 1)[0]

# Seleccionar 8 imágenes buenas y 8 con anomalías
num_samples = 8
selected_good_indices = good_indices[:num_samples] if len(good_indices) >= num_samples else good_indices
selected_anomaly_indices = anomaly_indices[:num_samples] if len(anomaly_indices) >= num_samples else anomaly_indices

# Concatenar todas las imágenes
all_test_images = torch.cat(test_images_all)
all_test_reconstructions = torch.cat(test_reconstructions_all)

# Seleccionar las 16 imágenes (8 buenas + 8 con anomalías)
selected_indices = np.concatenate([selected_good_indices, selected_anomaly_indices])
sample_images = all_test_images[selected_indices]
sample_reconstructions = all_test_reconstructions[selected_indices]
sample_labels = test_labels[selected_indices]

print(f'Imágenes seleccionadas: {len(selected_good_indices)} buenas + {len(selected_anomaly_indices)} con anomalías = {len(sample_images)} total')

# Loggear estadísticas de scores en wandb
wandb_logger.experiment.log({
    "eval/min_score": float(anomaly_scores.min()),
    "eval/max_score": float(anomaly_scores.max()),
    "eval/mean_score": float(anomaly_scores.mean()),
    "eval/std_score": float(anomaly_scores.std())
})

### 4.3. Cálculo de Métricas y Curva ROC

Esta celda calcula las métricas de evaluación:
- **AUC-ROC**: Área bajo la curva ROC (mientras más cercano a 1.0, mejor)
- **Average Precision**: Precisión promedio (mientras más cercano a 1.0, mejor)
- Genera la curva ROC para visualización

In [ ]:
auc_score = roc_auc_score(test_labels, anomaly_scores)
ap_score = average_precision_score(test_labels, anomaly_scores)
print(f'AUC-ROC: {auc_score:.4f}')
print(f'Average Precision: {ap_score:.4f}')

# Curva ROC
fpr, tpr, thresholds = roc_curve(test_labels, anomaly_scores)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_score:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
roc_curve_path = os.path.join(DRIVE_BASE_PATH, 'roc_curve.png')
plt.savefig(roc_curve_path)
plt.show()

# Loggear métricas y curva ROC en wandb
wandb_logger.experiment.log({
    "eval/auc_roc": auc_score,
    "eval/average_precision": ap_score,
    "eval/roc_curve": wandb.Image(roc_curve_path)
})

# Crear curva ROC interactiva en wandb
# Nota: wandb.plot.roc_curve requiere que los scores sean probabilidades
# Normalizamos los scores para que estén en [0, 1]
normalized_scores = (anomaly_scores - anomaly_scores.min()) / (anomaly_scores.max() - anomaly_scores.min() + 1e-8)

# wandb.plot.roc_curve necesita un formato específico: y_probas debe ser un array 2D de forma (n_samples, n_classes)
# Para binario, necesitamos [prob_clase_0, prob_clase_1] donde prob_clase_1 = normalized_scores
try:
    # Crear array de probabilidades con forma (n_samples, 2)
    y_probas = np.column_stack([1 - normalized_scores, normalized_scores])
    wandb_logger.experiment.log({
        "eval/roc_curve_interactive": wandb.plot.roc_curve(test_labels,
                                                           y_probas,
                                                           labels=["Normal", "Anomalía"],
                                                           classes_to_plot=[1])
    })
except Exception as e:
    print(f"Error al crear curva ROC interactiva en wandb: {e}")
    print("La curva ROC estática ya fue guardada correctamente.")

## 5. Visualización de Resultados

Esta sección contiene todas las visualizaciones de los resultados del modelo entrenado.


### 5.1. Reconstrucciones del Set de Prueba

Esta celda visualiza las reconstrucciones del set de prueba:
- **8 imágenes buenas**: Originales y sus reconstrucciones
- **8 imágenes con anomalías**: Originales y sus reconstrucciones
- Total: 16 imágenes (32 paneles: original + reconstrucción)

**Función `denormalize`**: Convierte las imágenes normalizadas de [-1, 1] a [0, 1] para visualización correcta.

In [ ]:
def denormalize(tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]):
    """Desnormaliza un tensor de imagen de [-1, 1] a [0, 1] para visualización"""
    # Desnormalizar: x = (x_norm * std) + mean
    # Para normalización [-1, 1]: x_norm = (x - 0.5) / 0.5, entonces x = (x_norm * 0.5) + 0.5
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor.clamp_(0, 1)

# Visualizar reconstrucciones del set de prueba (16 imágenes: 8 buenas + 8 con anomalías)
fig, axes = plt.subplots(4, 8, figsize=(20, 10))
axes = axes.flatten()
test_reconstruction_images = []

# Visualizar 8 imágenes buenas primero
for i in range(8):
    if i < len(sample_images):
        idx = i
        img = denormalize(sample_images[idx].clone())
        recon = denormalize(sample_reconstructions[idx].clone())

        # Imagen original
        axes[i*2].imshow(img.permute(1, 2, 0))
        axes[i*2].set_title(f'Buenas - Original {i+1}')
        axes[i*2].axis('off')

        # Reconstrucción
        axes[i*2+1].imshow(recon.permute(1, 2, 0))
        axes[i*2+1].set_title(f'Buenas - Reconstrucción {i+1}')
        axes[i*2+1].axis('off')

        # Preparar para wandb
        img_np = (img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        recon_np = (recon.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        test_reconstruction_images.append(wandb.Image(img_np, caption=f"Buenas - Original {i+1}"))
        test_reconstruction_images.append(wandb.Image(recon_np, caption=f"Buenas - Reconstrucción {i+1}"))

# Visualizar 8 imágenes con anomalías
for i in range(8):
    idx = 8 + i
    if idx < len(sample_images):
        img = denormalize(sample_images[idx].clone())
        recon = denormalize(sample_reconstructions[idx].clone())

        # Imagen original
        axes[16 + i*2].imshow(img.permute(1, 2, 0))
        axes[16 + i*2].set_title(f'Anomalía - Original {i+1}')
        axes[16 + i*2].axis('off')

        # Reconstrucción
        axes[16 + i*2+1].imshow(recon.permute(1, 2, 0))
        axes[16 + i*2+1].set_title(f'Anomalía - Reconstrucción {i+1}')
        axes[16 + i*2+1].axis('off')

        # Preparar para wandb
        img_np = (img.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        recon_np = (recon.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        test_reconstruction_images.append(wandb.Image(img_np, caption=f"Anomalía - Original {i+1}"))
        test_reconstruction_images.append(wandb.Image(recon_np, caption=f"Anomalía - Reconstrucción {i+1}"))

plt.tight_layout()
test_recon_path = os.path.join(DRIVE_BASE_PATH, 'test_reconstructions.png')
plt.savefig(test_recon_path)
plt.show()

# Loggear reconstrucciones del set de prueba en wandb
wandb_logger.experiment.log({
    "test/reconstructions": test_reconstruction_images,
    "test/reconstructions_grid": wandb.Image(test_recon_path)
})
print("Reconstrucciones del set de prueba (16 imágenes: 8 buenas + 8 con anomalías) guardadas en wandb")

### 5.2. Distribución de Scores de Anomalía

Esta celda visualiza la distribución de los scores de anomalía:
- Histograma comparativo entre imágenes normales y con anomalías
- Boxplot de scores por clase

In [ ]:
normal_scores = anomaly_scores[test_labels == 0]
anomaly_scores_plot = anomaly_scores[test_labels == 1]

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.hist(normal_scores, bins=50, alpha=0.7, label='Normal', color='green')
plt.hist(anomaly_scores_plot, bins=50, alpha=0.7, label='Anomalía', color='red')
plt.xlabel('Anomaly Score (MSE)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Scores de Anomalía')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.boxplot([normal_scores, anomaly_scores_plot], tick_labels=['Normal', 'Anomalía'])
plt.ylabel('Anomaly Score (MSE)')
plt.title('Boxplot de Scores')
plt.grid(True, alpha=0.3)
plt.tight_layout()

score_dist_path = os.path.join(DRIVE_BASE_PATH, 'score_distribution.png')
plt.savefig(score_dist_path)
plt.show()

# Loggear distribución de scores en wandb
wandb_logger.experiment.log({
    "eval/score_distribution": wandb.Image(score_dist_path),
    "eval/normal_scores_mean": float(normal_scores.mean()),
    "eval/normal_scores_std": float(normal_scores.std()),
    "eval/anomaly_scores_mean": float(anomaly_scores_plot.mean()),
    "eval/anomaly_scores_std": float(anomaly_scores_plot.std())
})

# Crear histograma interactivo en wandb
wandb_logger.experiment.log({
    "eval/scores_histogram": wandb.Histogram(anomaly_scores, num_bins=50)
})

### 5.3. Análisis de Mejores y Peores Reconstrucciones

Esta celda identifica las imágenes con menor y mayor error de reconstrucción, lo que ayuda a entender el comportamiento del modelo.

In [ ]:

sorted_indices = np.argsort(anomaly_scores)

print("Top 5 imágenes con menor error (más normales):")
for idx in sorted_indices[:5]:
    print(f"  {test_paths[idx]} - Score: {anomaly_scores[idx]:.4f}")

print("\nTop 5 imágenes con mayor error (más anómalas):")
for idx in sorted_indices[-5:]:
    print(f"  {test_paths[idx]} - Score: {anomaly_scores[idx]:.4f}")


### 5.4. Histogramas de Error de Reconstrucción por Subclase de Defecto

Esta celda genera histogramas del error de reconstrucción entre clases normales y defectuosas para cada subclase de defecto del set de pruebas, según los requisitos del enunciado.


In [ ]:
# Función para obtener la subclase de defecto de una ruta
def get_defect_subclass(path, category_path):
    """Extrae la subclase de defecto de la ruta de la imagen"""
    test_path = os.path.join(category_path, 'test')
    if 'good' in path:
        return 'good'
    # Extraer el nombre de la subclase (directorio después de 'test/')
    relative_path = os.path.relpath(path, test_path)
    subclass = os.path.dirname(relative_path)
    return subclass if subclass else 'unknown'

# Obtener subclases de defecto para todas las imágenes de prueba
defect_subclasses = [get_defect_subclass(path, category_path) for path in test_paths_all]

# Crear diccionario con scores por subclase
scores_by_subclass = {}
for subclass, score in zip(defect_subclasses, anomaly_scores):
    if subclass not in scores_by_subclass:
        scores_by_subclass[subclass] = []
    scores_by_subclass[subclass].append(score)

# Crear histogramas para cada subclase
n_subclasses = len(scores_by_subclass)
n_cols = 3
n_rows = (n_subclasses + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 6 * n_rows))
axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes

for idx, (subclass, scores) in enumerate(sorted(scores_by_subclass.items())):
    ax = axes[idx]

    # Separar scores de 'good' y defectos
    if subclass == 'good':
        ax.hist(scores, bins=30, alpha=0.7, label='Normal (good)', color='green', edgecolor='black')
        ax.set_title(f'Histograma de Error - {subclass.capitalize()}\n(n={len(scores)} imágenes)')
    else:
        # Comparar con 'good' si existe
        if 'good' in scores_by_subclass:
            good_scores = scores_by_subclass['good']
            ax.hist(good_scores, bins=30, alpha=0.5, label='Normal (good)', color='green', edgecolor='black')
            ax.hist(scores, bins=30, alpha=0.7, label=f'Defecto ({subclass})', color='red', edgecolor='black')
            ax.set_title(f'Histograma de Error - {subclass}\n(n={len(scores)} imágenes)')
        else:
            ax.hist(scores, bins=30, alpha=0.7, label=f'Defecto ({subclass})', color='red', edgecolor='black')
            ax.set_title(f'Histograma de Error - {subclass}\n(n={len(scores)} imágenes)')

    ax.set_xlabel('Error de Reconstrucción (MSE)')
    ax.set_ylabel('Frecuencia')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Ocultar ejes no utilizados
for idx in range(len(scores_by_subclass), len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
histograms_path = os.path.join(DRIVE_BASE_PATH, 'histogramas_por_subclase.png')
plt.savefig(histograms_path, dpi=150, bbox_inches='tight')
plt.show()

# Loggear histogramas en wandb
try:
    if 'wandb_logger' in globals() and wandb_logger is not None:
        wandb_logger.experiment.log({
            "eval/histograms_by_subclass": wandb.Image(histograms_path)
        })
        print("Histogramas por subclase guardados en wandb")
    else:
        print(f"Histogramas guardados en: {histograms_path}")
        print("Nota: wandb_logger no está definido. Ejecuta primero la celda de entrenamiento para habilitar el logging.")
except Exception as e:
    print(f"Error al loggear en wandb: {e}")
    print(f"Histogramas guardados en: {histograms_path}")

# Mostrar estadísticas por subclase
print("\n" + "="*80)
print("ESTADÍSTICAS DE ERROR POR SUBCLASE")
print("="*80)
for subclass in sorted(scores_by_subclass.keys()):
    scores = scores_by_subclass[subclass]
    print(f"\n{subclass}:")
    print(f"  Número de imágenes: {len(scores)}")
    print(f"  Error medio: {np.mean(scores):.4f}")
    print(f"  Error mediano: {np.median(scores):.4f}")
    print(f"  Error std: {np.std(scores):.4f}")
    print(f"  Error mínimo: {np.min(scores):.4f}")
    print(f"  Error máximo: {np.max(scores):.4f}")


### 5.5. Comparación de Funciones de Pérdida (Train/Val Loss)

Esta celda compara las funciones de pérdida mostrando las gráficas de pérdida de entrenamiento y validación entre las diferentes configuraciones, según los requisitos del enunciado.

**Nota**: Esta visualización requiere que se hayan ejecutado múltiples experimentos (celda 2.1) o que se hayan guardado los resultados de entrenamiento.


In [ ]:
# Comparación de funciones de pérdida
# Si se ejecutó la celda de experimentos automatizados, usar esos resultados
# Si no, usar los resultados del experimento actual

if 'experiment_results' in globals() and len(experiment_results) > 0:
    # Usar resultados de experimentos automatizados
    print("Usando resultados de experimentos automatizados...")

    # Crear gráficas de comparación de pérdidas train/val por arquitectura
    architectures = results_df['architecture'].unique()

    for arch in architectures:
        arch_data = results_df[results_df['architecture'] == arch]

        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Gráfica de pérdidas de entrenamiento
        ax1 = axes[0]
        for loss_func in LOSS_FUNCTIONS:
            exp_data = arch_data[arch_data['loss_function'] == loss_func]
            if len(exp_data) > 0 and exp_data.iloc[0]['train_losses']:
                train_losses = exp_data.iloc[0]['train_losses']
                epochs = range(1, len(train_losses) + 1)
                ax1.plot(epochs, train_losses, label=loss_func, linewidth=2, marker='o', markersize=4)

        ax1.set_xlabel('Época')
        ax1.set_ylabel('Pérdida de Entrenamiento')
        ax1.set_title(f'Comparación de Pérdida de Entrenamiento - {arch}')
        ax1.legend(title='Función de Pérdida')
        ax1.grid(True, alpha=0.3)

        # Gráfica de pérdidas de validación
        ax2 = axes[1]
        for loss_func in LOSS_FUNCTIONS:
            exp_data = arch_data[arch_data['loss_function'] == loss_func]
            if len(exp_data) > 0 and exp_data.iloc[0]['val_losses']:
                val_losses = exp_data.iloc[0]['val_losses']
                # Filtrar valores None
                val_losses_clean = [v for v in val_losses if v is not None]
                if val_losses_clean:
                    epochs = range(1, len(val_losses_clean) + 1)
                    ax2.plot(epochs, val_losses_clean, label=loss_func, linewidth=2, marker='s', markersize=4)

        ax2.set_xlabel('Época')
        ax2.set_ylabel('Pérdida de Validación')
        ax2.set_title(f'Comparación de Pérdida de Validación - {arch}')
        ax2.legend(title='Función de Pérdida')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()
        comparison_loss_path = os.path.join(DRIVE_BASE_PATH, f'comparacion_loss_{arch}.png')
        plt.savefig(comparison_loss_path, dpi=150, bbox_inches='tight')
        plt.show()

        # Loggear en wandb
        try:
            if 'wandb_logger' in globals() and wandb_logger is not None:
                wandb_logger.experiment.log({
                    f"comparison/loss_comparison_{arch}": wandb.Image(comparison_loss_path)
                })
        except Exception as e:
            print(f"Error al loggear en wandb: {e}")

    print("\n✓ Comparación de funciones de pérdida completada")

else:
    # Si no hay experimentos automatizados, mostrar solo el experimento actual
    print("No se encontraron resultados de experimentos automatizados.")
    print("Para ver la comparación completa, ejecuta la celda 2.1 (Ejecución Automatizada de Experimentos).")
    print("\nMostrando solo el experimento actual:")

    if 'train_losses' in globals() and train_losses:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))

        # Gráfica de pérdida de entrenamiento
        epochs = range(1, len(train_losses) + 1)
        axes[0].plot(epochs, train_losses, label=cfg.trainer.loss_function, linewidth=2, marker='o', markersize=4)
        axes[0].set_xlabel('Época')
        axes[0].set_ylabel('Pérdida de Entrenamiento')
        axes[0].set_title(f'Pérdida de Entrenamiento - {cfg.model.architecture}')
        axes[0].legend(title='Función de Pérdida')
        axes[0].grid(True, alpha=0.3)

        # Gráfica de pérdida de validación (si está disponible)
        if 'val_losses' in globals() and val_losses:
            val_losses_clean = [v for v in val_losses if v is not None]
            if val_losses_clean:
                epochs_val = range(1, len(val_losses_clean) + 1)
                axes[1].plot(epochs_val, val_losses_clean, label=cfg.trainer.loss_function, linewidth=2, marker='s', markersize=4)
                axes[1].set_xlabel('Época')
                axes[1].set_ylabel('Pérdida de Validación')
                axes[1].set_title(f'Pérdida de Validación - {cfg.model.architecture}')
                axes[1].legend(title='Función de Pérdida')
                axes[1].grid(True, alpha=0.3)
        else:
            axes[1].text(0.5, 0.5, 'Pérdidas de validación no disponibles',
                        ha='center', va='center', transform=axes[1].transAxes)
            axes[1].set_title(f'Pérdida de Validación - {cfg.model.architecture}')

        plt.tight_layout()
        current_loss_path = os.path.join(DRIVE_BASE_PATH, f'loss_current_{cfg.model.architecture}_{cfg.trainer.loss_function}.png')
        plt.savefig(current_loss_path, dpi=150, bbox_inches='tight')
        plt.show()

        # Loggear en wandb
        try:
            if 'wandb_logger' in globals() and wandb_logger is not None:
                wandb_logger.experiment.log({
                    f"train/loss_comparison_current": wandb.Image(current_loss_path)
                })
        except Exception as e:
            print(f"Error al loggear en wandb: {e}")
    else:
        print("No hay datos de pérdidas disponibles para visualizar.")


## 6. Guardar Modelo

Esta sección guarda el modelo entrenado y finaliza la sesión de WandB.


### 6.1. Guardado del Modelo y Finalización

Esta celda:
- Guarda el estado del modelo entrenado (pesos, optimizador, métricas)
- Guarda el modelo en WandB para acceso posterior
- Finaliza la sesión de WandB

In [ ]:
model_save_path = os.path.join(DRIVE_BASE_PATH, 'modelo_autoencoder.pth')
torch.save({
    'model_state_dict': lightning_model.model.state_dict(),
    'optimizer_state_dict': lightning_model.optimizers().state_dict(),
    'train_losses': train_losses,
    'auc_score': auc_score,
    'ap_score': ap_score
}, model_save_path)
print(f'Modelo guardado en: {model_save_path}')

# Guardar modelo en wandb
try:
    if 'wandb_logger' in globals() and wandb_logger is not None:
        wandb_logger.experiment.save(model_save_path)
        print("✓ Modelo guardado en WandB")
    else:
        print("ℹ️ wandb_logger no está disponible. El modelo se guardó localmente.")
except Exception as e:
    print(f"ℹ️ Error al guardar modelo en WandB: {e}")
    print("   El modelo se guardó localmente.")

# NOTA: No finalizamos WandB aquí para permitir que las celdas de visualización posteriores puedan loggear
# WandB se finalizará al final del notebook (ver celda final)

## Notas Finales

- El modelo ha sido entrenado solo con imágenes normales (good)
- Las anomalías se detectan mediante el error de reconstrucción
- Puedes ajustar el umbral de detección según tus necesidades
- Para cargar el modelo guardado, usa: `torch.load(model_save_path)`

### Weights & Biases

- Todas las métricas de entrenamiento y evaluación se registran automáticamente en wandb
- Revisa tus resultados en: https://wandb.ai
- El proyecto se llama: `tarea05-anomaly-detection`
- Cada run se identifica por la categoría utilizada

### Métricas Registradas

- **Entrenamiento**: pérdida por batch, pérdida por época, learning rate
- **Evaluación**: AUC-ROC, Average Precision, estadísticas de scores
- **Visualizaciones**: curva ROC, reconstrucciones, distribución de scores
- **Modelo**: se guarda automáticamente en wandb


### 6.2. Finalización de WandB

Esta celda finaliza la sesión de WandB después de que todas las visualizaciones y métricas hayan sido registradas.

**Nota**: Esta celda debe ejecutarse al final de todo el proceso para cerrar correctamente la sesión de WandB.


In [ ]:
# Finalizar WandB run (solo si está activo)
try:
    # Intentar finalizar usando wandb_logger
    if 'wandb_logger' in globals() and wandb_logger is not None:
        try:
            wandb_logger.experiment.finish()
            print("✓ WandB run finalizado correctamente")
        except Exception as e:
            if "finished" in str(e).lower() or "is finished" in str(e).lower():
                print("ℹ️ El run de WandB ya estaba finalizado")
            else:
                print(f"⚠️ Error al finalizar wandb_logger: {e}")
    
    # Intentar finalizar usando wandb directamente (para experimentos automatizados)
    if 'wandb' in globals() and wandb.run is not None:
        try:
            if not wandb.run.settings._disable:
                wandb.finish()
                print("✓ WandB run finalizado correctamente")
        except Exception as e:
            if "finished" in str(e).lower() or "is finished" in str(e).lower():
                print("ℹ️ El run de WandB ya estaba finalizado")
            else:
                print(f"⚠️ Error al finalizar wandb: {e}")
    
    print("\n" + "="*80)
    print("PROCESO COMPLETADO")
    print("="*80)
    print("Revisa tus métricas y visualizaciones en: https://wandb.ai")
    print(f"Proyecto: tarea05-anomaly-detection")
    print("="*80)
    
except Exception as e:
    print(f"⚠️ Error al finalizar WandB: {e}")
    print("   Puedes revisar tus métricas en: https://wandb.ai")
